In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = './data/OccupationProjections.xls'

In [65]:
df = pd.read_excel(file_path, skiprows=2)
df.columns=['OccupationLevel',
            'SkillLevel',
            'OccupationCode',
            'Occupation',
            'EmploymentLevel_May2018',
            'EmploymentLevel_May2023',
            'EmploymenGrowth_2018_to_2023',
            'EmploymenGrowth_2018_to_2023_percentage']

df.drop(df.tail(3).index,inplace=True) # Last few rows are not useful

In [60]:
df.head()

,OccupationLevel,SkillLevel,OccupationCode,Occupation,EmploymentLevel_May2018,EmploymentLevel_May2023,EmploymenGrowth_2018_to_2023,EmploymenGrowth_2018_to_2023_percentage
0,1,,1,MANAGERS,1561.630105,1671.502836,109.872731,7.035772
1,2,,11,"Chief Executives, General Managers and Legisla...",109.718797,114.566984,4.848187,4.418739
2,3,,111,"Chief Executives, General Managers and Legisla...",109.718797,114.566984,4.848187,4.418739
3,4,1,1111,Chief Executives and Managing Directors,62.824880,68.618668,5.793789,9.222124
4,4,1,1112,General Managers,42.125263,41.095188,-1.030075,-2.445267


# Things to make

### A thing that gives you every child of a given occupation code

### A thing that gives you the skill level of a given 4th level occupation code

# Occupation code to skill level

In [61]:
# Note that some entries have multiple values. These are preserved as strings, e.g. `1,2,3,4,5`
anzsco_to_skill_level = df[df['OccupationLevel']==4][['SkillLevel','OccupationCode']].set_index('OccupationCode')['SkillLevel'].to_dict()

# Map between occupation codes

In [62]:
occupations_level0 = {}
current_level1 = None
current_level2 = None
current_level3 = None
current_level4 = None

for index, row in df.iterrows():
    if len(str(row['OccupationCode']))==1:
        current_level1 = row['OccupationCode']
        
        # New Level1
        occupations_level0[current_level1] = {'Name': row['Occupation'],
                                              'Children':{}}
        continue
    if len(str(row['OccupationCode']))==2:
        current_level2 = row['OccupationCode']
        
        # New Level2
        occupations_level0[current_level1]['Children'][current_level2] = {'Name': row['Occupation'],
                                                                          'Children': {}}
        continue
    if len(str(row['OccupationCode']))==3:
        current_level3 = row['OccupationCode']
        
        
        # New Level3
        occupations_level0[current_level1]['Children'][current_level2]['Children'][current_level3] = {'Name': row['Occupation'],
                                                                                                      'Children': {}}
        continue
    if len(str(row['OccupationCode']))==4:
        # New Level4
        occupations_level0[current_level1]['Children'][current_level2]['Children'][current_level3]['Children'][row['OccupationCode']] = row['Occupation']
        
        continue
    


In [67]:
occupations_level0[4]['Children'][42]

{'Name': 'Carers and Aides',
 'Children': {421: {'Name': 'Child Carers',
   'Children': {4211: 'Child Carers'}},
  422: {'Name': 'Education Aides', 'Children': {4221: 'Education Aides'}},
  423: {'Name': 'Personal Carers and Assistants',
   'Children': {4231: 'Aged and Disabled Carers',
    4232: 'Dental Assistants',
    4233: 'Nursing Support and Personal Care Workers',
    4234: 'Special Care Workers',
    4230: 'Personal Carers and Assistants nfd'}},
  420: {'Name': 'Carers and Aides nfd',
   'Children': {4200: 'Carers and Aides nfd'}}}}